In [0]:
!pip install -r /Workspace/Users/kevin.zhang@vgw.co/AI_image_anomaly_detection/ai-toolkit/requirements.txt
!pip install diffusers
!pip install dwpose
!pip install -U accelerate
!pip install -U transformers
!pip install protobuf==3.20.0

dbutils.library.restartPython()

In [0]:
from huggingface_hub import login

login(token = 'your DB token')

In [0]:
import os
import sys
sys.path.append('/Workspace/Users/kevin.zhang@vgw.co/AI_image_anomaly_detection/ai-toolkit')
from toolkit.job import run_job
from collections import OrderedDict
from PIL import Image
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [0]:
from huggingface_hub import login

login(token = 'your DB token')

In [0]:
import pandas as pd

raw_data=pd.read_csv('your csv file with image file names and expected text').dropna()
raw_data['idx']=[f.split('-')[-1][:-3] for f in raw_data['filename']]
raw_data

In [0]:
import os
from PIL import Image
import shutil
import pandas as pd

data_list=[]
for img in os.listdir('your training_image path'):
    if img.find('ocr')>=0:
        data_list.append(img.split('-')[-1][:-3])

In [0]:
data_list_df=pd.DataFrame({'idx': data_list}, index=range(len(data_list)))
data_list_df.shape

In [0]:
df=raw_data.merge(data_list_df, how='inner')
df

In [0]:
df['prompt'] = ['A person is holding a note in front of his face and the note says "' + e + '"'  for e in df.expected]
df['prompt'].tolist()[0]

In [0]:
df.to_csv('flux_finetuning_input.csv', index=False)

In [0]:
for d in df[['filename', 'prompt']].values.tolist():
    with open('/Volumes/dsp_lab/dsp_sandbox_tbls/data/ordered_files/' + d[0][:-4] + ".txt", "w") as f:
        f.write(d[1])

In [0]:
%sh
accelerate launch --num_processes=6 /Workspace/Users/kevin.zhang@vgw.co/AI_image_anomaly_detection/ai-toolkit/run_job.py

In [0]:
from diffusers import AutoPipelineForText2Image, FluxPipeline
import torch

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16, device_map="balanced")

In [0]:
pipe.load_lora_weights('/Volumes/dsp_lab/dsp_sandbox_tbls/data/flux_finetuning_output_3/my_first_flux_lora_v1', weight_name='my_first_flux_lora_v1.safetensors', adapter_name='lora')
pipe.set_adapters('lora')
pipe.fuse_lora(adapter_names=['lora'])

In [0]:
prompt = 'A person is holding a note in front of his face and the note says "7821731022696 seven eight two one seven three one zero two two six nine six I wish to receive sweep coins to participate in the promotion offered by Vanilla By submitting this request, I hereby declare that I have read, understood and agree to be bound by Vanilla Terms and Conditions and Sweeps rules"'
image = pipe(
    prompt,
    height=512,
    width=512,
    guidance_scale=50,
    num_inference_steps=300,
    max_sequence_length=512,
    # generator=torch.Generator("cuda").manual_seed(0)
).images[0]
image